<h1 style='color: gold; text-align: center; font-family: cursive;font-size: 30px;'>NLP Approach</h1>

<h1 style='color: rgb(77, 229, 240); text-align: left; font-family: cursive;font-size: 20px;'>importing liabrary</h1>

In [1]:
# basic modules
import pandas as pd
import numpy as np

#nlp modules
import nltk 
from sklearn.feature_extraction.text import CountVectorizer 
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re

# modules to read pdf files
from io import StringIO
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage

# ignore warning
import warnings 
warnings.simplefilter('ignore') 

# moduels require for machine learning
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor 

# for evaluation of model
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import mean_squared_error

#other modules
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split 

<h1 style='color: rgb(77, 229, 240); text-align: left; font-family: cursive;font-size: 20px;'>data loading</h1>

In [2]:
'''
### Read in PDF complicated text using Pdfminer
'''      


def pdf_text_reader(pdf_file_name, pages=None):
    if pages:
        pagenums = set(pages)
    else:
        pagenums = set()

    ## 1) Initiate the Pdf text converter and interpreter
    textOutput = StringIO()
    manager = PDFResourceManager()
    converter = TextConverter(manager, textOutput, laparams=LAParams())
    interpreter = PDFPageInterpreter(manager, converter)

    ## 2) Extract text from file using the interpreter
    infile = open(pdf_file_name, 'rb')
    for page in PDFPage.get_pages(infile, pagenums):
        interpreter.process_page(page)        
    infile.close()
    
    ## 3) Extract the paragraphs and close the connections
    paras = textOutput.getvalue()   
    converter.close()
    textOutput.close
    
    return paras


In [3]:
def pdf_reader(file_name):

    file = pdf_text_reader(file_name, pages=[0]) #start from page no 0
    page_no=1
    while True:
      another_page = pdf_text_reader(file_name, pages=[page_no])

      if(len(another_page)==0): #add pages untill pages ends
            break 
      file = file+another_page
      page_no+=1
    #converting text to lower case 
    #file = [i.lower() for i in file.split('\n') if i] #splliting context of file for better handling 
    return file #it returns list containing lines of pdf pages as strings

In [4]:
#training data
train = pd.read_csv('dataset/train.csv')
print("Train Data :") 
train.head(3) #printing first 3 entries


Train Data :


,CandidateID,Match Percentage
0,candidate_011,13.60
1,candidate_113,36.63
2,candidate_123,54.93


In [5]:
#testing data 
test = pd.read_csv('dataset/test.csv')  
print("Test Data :") 
test.head(3) #printing first 3 entries 

Test Data :


,CandidateID
0,candidate_014
1,candidate_098
2,candidate_075


In [6]:
#sort dataframe based on candidateId
train.sort_values(by ='CandidateID', inplace =True)

#reset index
train.reset_index(inplace=True)
train.drop('index',axis=1,inplace=True)

#print sorted train dataset
train.head() 

,CandidateID,Match Percentage
0,candidate_000,13.70
1,candidate_001,40.09
2,candidate_002,48.91
3,candidate_003,36.89
4,candidate_006,44.96


In [7]:
#sort dataframe based on candidateId
test.sort_values(by ='CandidateID', inplace =True)

#reset index
test.reset_index(inplace=True)
test.drop('index',axis=1,inplace=True)

#print sorted test dataset
test.head() 

,CandidateID
0,candidate_004
1,candidate_005
2,candidate_014
3,candidate_016
4,candidate_017


In [8]:
train["CV"] = 0
for i in range(90):
    train["CV"][i] = pdf_reader("dataset/trainResumes/"+str(train["CandidateID"][i])+".pdf")

train.head(3)     

,CandidateID,Match Percentage,CV
0,candidate_000,13.70,JACOB SMITH\n\nF R E S H E R\n\nPERSONAL PROFI...
1,candidate_001,40.09,Brianna Williams\n\nJ UNI OR DEVELOPER\n\nWor...
2,candidate_002,48.91,MASON QUADRADO\n\nASSOCIATE ANALYST\n\nABOUT\n...


In [9]:
test["CV"] = 0
for i in range(60):
    test["CV"][i] = pdf_reader("dataset/testResumes/"+str(test["CandidateID"][i])+".pdf")

test.head(3)    

,CandidateID,CV
0,candidate_004,OLIVIA SANTOS\n\nCONSULTANT ANALYST\n\nEXECUTI...
1,candidate_005,ARMIN FITZGERALD\n\nD A T A M A N ...
2,candidate_014,GRACE BAILRY\n\nM A C H I N E L E A R N I N ...


In [10]:
jd = pdf_reader("dataset/Job description.pdf")
jd=jd.replace("\n"," ")
jd=jd.lower()
jd[0:250] #print first 5 lines

'machine learning engineering  13585abc knowledge and innovation  what you’ll do  you will focus on researching, building, and designing self-running artificial intelligence (ai) systems to automate predictive models. you are responsible to design and'

In [11]:
#remove spaces \n as well as convert it to lower case
for i in range(90):
    train["CV"][i]=train["CV"][i].replace("\n"," ")
    train["CV"][i]=train["CV"][i].lower()

train.head()

,CandidateID,Match Percentage,CV
0,candidate_000,13.70,jacob smith f r e s h e r personal profile i...
1,candidate_001,40.09,brianna williams j uni or developer work ex...
2,candidate_002,48.91,mason quadrado associate analyst about certi...
3,candidate_003,36.89,rodney michaels a s s o c i a t e s o f t w...
4,candidate_006,44.96,j e n n i f e r a r m s t r o n g p r o f i ...


In [12]:
#remove spaces \n as well as convert it to lower case
for i in range(60):
    test["CV"][i]=test["CV"][i].replace("\n"," ")
    test["CV"][i]=test["CV"][i].lower()

test.head()

,CandidateID,CV
0,candidate_004,olivia santos consultant analyst executive s...
1,candidate_005,armin fitzgerald d a t a m a n ...
2,candidate_014,grace bailry m a c h i n e l e a r n i n g ...
3,candidate_016,amelia martin data science major work experi...
4,candidate_017,jonathan hill j u n i o r m a c h i n e l...


In [13]:
test.CV[2]

"grace bailry  m a c h i n e   l e a r n i n g   d e v e l o p m e n t   a n d   d e p l o y m e n t    personal profile i am a software engineer who has worked on several computer vision projects and their end to end deployment. i am looking to join as a machine learning developer to closely work on problems and increase my grasp over the domain.   work background  eximius design pvt ltd company software development and deployment engineer apr 2019 to till date  implementing multiple components of a face identification system which include face alignment and landmark detection. improving the validation pipeline of face recognition: collected data under different lighting conditions. performed image manipulation operations on data to increase the accuracy of the pipeline.  skills  software engineer/developer, software development, project management, machine learning, computer vision, opencv, python, c++, ruby, learning algorithm.   education  b.tech in electronics & computers from dvr

<h1 style='color: rgb(77, 229, 240); text-align: left; font-family: cursive;font-size: 20px;'>collection of all cv</h1>

In [14]:
cv=list(train.CV)
len(cv)
cv=cv+list(test.CV)
len(cv)
cv=cv+[jd]
len(cv)

151

<h1 style='color: rgb(77, 229, 240); text-align: left; font-family: cursive;font-size: 20px;'>creating corpus</h1>

In [15]:
corpus = []
ps = PorterStemmer()

<h1 style='color: rgb(77, 229, 240); text-align: left; font-family: cursive;font-size: 20px;'>tokanize</h1>

In [16]:
for i in range(0,151):
    #Regular Expresion
    '''
    Replace email addresses with 'emailaddr'
    Replace URLs with 'httpaddr'
    Replace money symbols with 'moneysymb'
    Replace phone numbers with 'phonenumbr'
    Replace numbers with 'numbr'
    '''
    msg = cv[i] 
    msg = re.sub('\b[\w\-.]+?@\w+?\.\w{2,4}\b', 'emailaddr', cv[i])
    msg = re.sub('(http[s]?\S+)|(\w+\.[A-Za-z]{2,4}\S*)', 'httpaddr', cv[i])
    msg = re.sub('([A-Z]{3}|[A-Z]?[\$€¥])?\s?(\d{1,3}((,\d{1,3})+)?(.\d{1,3})?(.\d{1,3})?(,\d{1,3})?)', 'moneysymb', cv[i])
    msg = re.sub('\b(\+\d{1,2}\s)?\d?[\-(.]?\d{3}\)?[\s.-]?\d{3}[\s.-]?\d{4}\b', 'phonenumbr', cv[i])
    msg = re.sub('\d+(\.\d+)?', 'numbr', cv[i])

    ''' Remove all punctuations '''
    msg = re.sub('[^\w\d\s]', ' ', cv[i])
    
 

    
    # Each word to lower case
    msg = msg.lower()    

    
    # Splitting words to Tokenize
    msg = msg.split()    

    
    # Stemming with PorterStemmer handling Stop Words
    msg = [ps.stem(word) for word in msg if not word in set(stopwords.words('english'))]

    
    # preparing Messages with Remaining Tokens
    msg = ' '.join(msg)

    
    # Preparing WordVector Corpus
    corpus.append(msg)


<h1 style='color: rgb(77, 229, 240); text-align: left; font-family: cursive;font-size: 20px;'>creating vectors</h1>

In [17]:
cv = CountVectorizer()
x = cv.fit_transform(corpus).toarray() 

In [18]:
y=list(train["Match Percentage"])

In [19]:
xtrain = x[:90]
xtest = x[90:150]

<h1 style='color: rgb(77, 229, 240); text-align: left; font-family: cursive;font-size: 20px;'>dscision tree = 56.17</h1>

In [20]:
sv = SVR()
sv.fit(xtrain,y)
svP=sv.predict(xtest)

res={
    "CandidateID":list(test.CandidateID),
    "Match Percentage":svP
}

res=pd.DataFrame(res)

res.to_csv("res.csv",index=False)